In [1]:
!nvidia-smi

Wed Mar  1 14:20:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.84       Driver Version: 460.84       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:04:00.0 Off |                  N/A |
| 20%   21C    P8     8W / 250W |    447MiB / 11178MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:06:00.0 Off |                  N/A |
| 20%   

In [12]:
import os
import torch
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, "0" to  "7" 
os.environ["CUDA_VISIBLE_DEVICES"]="3"
#torch.cuda.set_device(0)
n_gpu = torch.cuda.device_count()
print(n_gpu)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.get_device_name(0)


8


'GeForce GTX 1080 Ti'

In [3]:
import warnings 
warnings.filterwarnings('ignore')
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm


In [4]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation
#Set a seed
import random
import numpy as np
import torch
import datasets
import pandas as pd

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

from torch.utils.data import Dataset, DataLoader

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

[nltk_data] Downloading package punkt to /DATA/gupta92/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

set_seed(42)

In [6]:
#Loading dataset

In [7]:
pd.set_option('display.max_colwidth', None)

In [8]:
df = pd.read_csv('DATA/etoori_train.csv')
df.shape


(140000, 2)

In [9]:
df.rename(columns = {'enc_input':'input'}, inplace = True)
df.rename(columns = {'dec_input':'output'}, inplace = True)

In [10]:
df.head(10)

,input,output
0,परन्तु वे दोनों उन बातों को ज़्यादा समय तक अप हृदय में गाँठ बनाकर नहीं रखते थे ।,परन्तु वे दोनों उन बातों को ज़्यादा समय तक अपने हृदय में गाँठ बनाकर नहीं रखते थे ।
1,देश में हिन्दी को विस्थापित कर का षड़यंत्र चल रहा है जो चिंता का विषय है ।,देश में हिन्दी को विस्थापित करने का षड़यंत्र चल रहा है जो चिंता का विषय है ।
2,तीन साल पहले कातिलाना हमले के प्रकरण में एफआर लग जा के बाद डेरा प्रेमी इस बार दर्ज हुए मुकदमे में सभी अभियुक्तों की गिरफ्तारी की मांग को लेकर आंदोलनरत हैं ।,तीन साल पहले कातिलाना हमले के प्रकरण में एफआर लग जाने के बाद डेरा प्रेमी इस बार दर्ज हुए मुकदमे में सभी अभियुक्तों की गिरफ्तारी की मांग को लेकर आंदोलनरत हैं ।
3,रामायण रिविजिटेड अ टेल ऑफ लव एंड एडवेंचर नाम से सिंगापुर के पेरानांका म्यूजियम में प्रदर्शनी लगाई गई हैं ।,रामायण रिविजिटेड अ टेल ऑफ लव एंड एडवेंचर नाम से सिंगापुर के पेरानांका म्यूजियम में प्रदर्शनी लगाई गई है ।
4,तब तक के लिए हमें विराम ले की अनुमति दीजिए ।,तब तक के लिए हमें विराम लेने की अनुमति दीजिए ।
5,देव स्थेन तक सड़क बनाने की परियोजना चल रही है ।,देव स्थान तक सड़क बनाने की परियोजना चल रही है ।
6,वर्ड वाचर व प्रसिद्ध चित्रकार अनूप साह आरएनएस को बताया कि श्वेतकंठ चिलचिल पक्षी का वैज्ञानिक नाम ह्वाइट थ्रोटेट लाफिंग थ्रस है ।,वर्ड वाचर व प्रसिद्ध चित्रकार अनूप साह ने आरएनएस को बताया कि श्वेतकंठ चिलचिल पक्षी का वैज्ञानिक नाम ह्वाइट थ्रोटेट लाफिंग थ्रस है ।
7,दो दिन पहले ज्ञानजी पोस्ट ठेली जिसमें,दो दिन पहले ज्ञानजी ने पोस्ट ठेली जिसमें
8,दुर्घटना की स्थिति में बीमा कम्पनी द्वारा दी जा वाली राशि निर्धारित कर दी गई है ।,दुर्घटना की स्थिति में बीमा कम्पनी द्वारा दी जाने वाली राशि निर्धारित कर दी गई है ।
9,देर से जाऊंगा तो दुकां बंद हो जाएंगी ।,देर से जाऊंगा तो दुकानें बंद हो जाएंगी ।


In [11]:
from transformers import (
    T5ForConditionalGeneration, T5Tokenizer, 
    Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
  )

from torch.utils.data import Dataset, DataLoader

In [12]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")

In [13]:
tokenizer

PreTrainedTokenizerFast(name_or_path='google/muril-base-cased', vocab_size=197285, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [14]:
tokenizer.model_max_length

512

In [1]:
#First Initialize with t5-base, then have trained and saved t5_gec_hindi_muRIL_1, t5_gec_hindi_muRIL_2, t5_gec_hindi_muRIL_3

In [16]:

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

#model_name = 't5-base'
#tokenizer = T5Tokenizer.from_pretrained(model_name)
#model = T5ForConditionalGeneration.from_pretrained(model_name)

model_name = 't5_gec_hindi_muRIL_3'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu' 
model = T5ForConditionalGeneration.from_pretrained(model_name).to(torch_device)



In [17]:
def calc_token_len(example):
    return len(tokenizer(example).input_ids)

In [18]:
#Train - Test Split
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.10, shuffle=True)
train_df.shape, test_df.shape

((126000, 2), (14000, 2))

In [19]:
test_df['input_token_len'] = test_df['input'].apply(calc_token_len)

In [20]:
test_df.head()

,input,output,input_token_len
40665,अँगरेज़ी सहभाषा के रूप में व्यवहत्त होनी चाहिए थी परंतु आज भी उसकी वर्चस्व सर्वत्र स्थापित है ।,अँगरेज़ी सहभाषा के रूप में व्यवहत्त होनी चाहिए थी परंतु आज भी उसका वर्चस्व सर्वत्र स्थापित है ।,28
48520,तथे गौशाला का,तथा गौशाला का,6
138403,बच्चे खेल रहे है किलकारियाँ गूंज रही है ब्रह्माण्ड में,बच्चे खेल रहे हैं किलकारियाँ गूंज रही हैं ब्रह्माण्ड में,14
130079,धरने पर किसानो ने कहा कि अब समय आ गया हैं कि गन्ने की खेती किसानो के लिये नुकसान का सौदा हो गयी हैं ।,धरने पर किसानो ने कहा कि अब समय आ गया है कि गन्ने की खेती किसानो के लिये नुकसान का सौदा हो गयी है ।,29
50146,सड़क मलबे का ढेर होने से दुपहिया व चार पहिया वाहन चालकों को परेशान होना पड़ रहा हैं ।,सड़क मलबे का ढेर होने से दुपहिया व चार पहिया वाहन चालकों को परेशान होना पड़ रहा है ।,23


In [21]:
test_df['input_token_len'].describe()

count    14000.000000
mean        21.935000
std         12.422581
min          4.000000
25%         14.000000
50%         19.000000
75%         27.000000
max        308.000000
Name: input_token_len, dtype: float64

In [22]:
#We will use a token length of 64 since it will cover the vast majority of examples
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [23]:
test_dataset

Dataset({
    features: ['input', 'output', 'input_token_len', '__index_level_0__'],
    num_rows: 14000
})

In [24]:
#Load the Dataset
from torch.utils.data import Dataset, DataLoader
class GrammarDataset(Dataset):
    def __init__(self, dataset, tokenizer,print_text=False):         
        self.dataset = dataset
        self.pad_to_max_length = False
        self.tokenizer = tokenizer
        self.print_text = print_text
        self.max_len = 64
  
    def __len__(self):
        return len(self.dataset)


    def tokenize_data(self, example):
        input_, target_ = example['input'], example['output']

        # tokenize inputs
        tokenized_inputs = tokenizer(input_, pad_to_max_length=self.pad_to_max_length, 
                                            max_length=self.max_len,
                                            return_attention_mask=True)
    
        tokenized_targets = tokenizer(target_, pad_to_max_length=self.pad_to_max_length, 
                                            max_length=self.max_len,
                                            return_attention_mask=True)

        inputs={"input_ids": tokenized_inputs['input_ids'],
            "attention_mask": tokenized_inputs['attention_mask'],
            "labels": tokenized_targets['input_ids']
        }
        
        return inputs

  
    def __getitem__(self, index):
        inputs = self.tokenize_data(self.dataset[index])
        
        if self.print_text:
            for k in inputs.keys():
                print(k, len(inputs[k]))

        return inputs

In [25]:
dataset = GrammarDataset(test_dataset, tokenizer, True)
print(dataset[10])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


input_ids 27
attention_mask 27
labels 27
{'input_ids': [104, 75159, 1154, 7899, 106524, 3768, 1169, 16015, 1250, 2316, 1419, 1123, 7132, 64219, 2115, 1125, 30361, 1438, 1228, 106524, 1117, 10171, 1194, 1254, 7544, 492, 105], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [104, 75159, 1154, 7899, 106524, 3768, 1169, 16015, 1250, 2316, 1243, 1123, 7132, 64219, 2115, 1125, 30361, 1438, 1228, 106524, 1117, 10171, 1194, 1254, 7544, 492, 105]}


In [26]:
#Define Evaluator
#!pip install rouge_score

In [27]:
from datasets import load_metric
rouge_metric = load_metric("rouge")

In [28]:
#Train Model
#data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding='longest', return_tensors='pt')
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding='longest', return_tensors='pt')


In [30]:
# defining training related arguments
batch_size = 8
'''
args = Seq2SeqTrainingArguments(output_dir="models/hindi/T5_muRIL_3",
                        evaluation_strategy="epoch",
                        per_device_train_batch_size=batch_size,
                        per_device_eval_batch_size=batch_size,
                        learning_rate=2e-5,
                        num_train_epochs=2,
                        weight_decay=0.1,
                        save_total_limit=2,
                        predict_with_generate=True,
                        fp16 = True,
                        gradient_accumulation_steps = 6,
                        #eval_steps = 500,
                        #save_steps = 2000,
                        #dataloader_num_workers= 4,
                        load_best_model_at_end=True,
                        logging_dir="logs/hindi/T5_muRIL_3")
'''

args = Seq2SeqTrainingArguments(
    output_dir="models/hindi/T5_muRIL",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.1,
    save_steps=10000,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=True,
    #logging_dir="logs/hindi/T5_muRIL_3"

)

In [31]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
#os.environ["CUDA_LAUNCH_BLOCKING"] = "false"
os.environ['TRANSFORMERS_NO_ADVISORY_WARNINGS'] = 'true'

In [32]:
import torch
#torch.cuda.empty_cache()

In [33]:
# defining trainer using 🤗
trainer = Seq2SeqTrainer(model=model, 
                args=args, 
                train_dataset= GrammarDataset(train_dataset, tokenizer),
                eval_dataset=GrammarDataset(test_dataset, tokenizer),
                tokenizer=tokenizer,
                data_collator=data_collator)
                #compute_metrics=compute_metrics)

Using cuda_amp half precision backend


In [34]:
#import torch
#torch.cuda.empty_cache()

In [35]:
#Wandb API Key: bcdbdd5ee9d76a20c90b5f2a246eb45f14b341a9
import os
os.environ["WANDB_DISABLED"] = "true"     #Disabling Wandb
#model.resize_token_embeddings(len(tokenizer))
trainer.train()

***** Running training *****
  Num examples = 126000
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 78750
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch,Training Loss,Validation Loss
1,0.524600,0.324090
2,0.444900,0.240954
3,0.391500,0.194559
4,0.356900,0.176006
5,0.348300,0.169690


Saving model checkpoint to models/hindi/T5_muRIL/checkpoint-10000
Configuration saved in models/hindi/T5_muRIL/checkpoint-10000/config.json
Model weights saved in models/hindi/T5_muRIL/checkpoint-10000/pytorch_model.bin
tokenizer config file saved in models/hindi/T5_muRIL/checkpoint-10000/tokenizer_config.json
Special tokens file saved in models/hindi/T5_muRIL/checkpoint-10000/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 14000
  Batch size = 8
Saving model checkpoint to models/hindi/T5_muRIL/checkpoint-20000
Configuration saved in models/hindi/T5_muRIL/checkpoint-20000/config.json
Model weights saved in models/hindi/T5_muRIL/checkpoint-20000/pytorch_model.bin
tokenizer config file saved in models/hindi/T5_muRIL/checkpoint-20000/tokenizer_config.json
Special tokens file saved in models/hindi/T5_muRIL/checkpoint-20000/special_tokens_map.json
Saving model checkpoint to models/hindi/T5_muRIL/checkpoint-30000
Configuration saved in models/hindi/T5_muRIL/checkpoint

TrainOutput(global_step=78750, training_loss=0.4303435190352183, metrics={'train_runtime': 23896.2568, 'train_samples_per_second': 26.364, 'train_steps_per_second': 3.295, 'total_flos': 2.96756122281984e+16, 'train_loss': 0.4303435190352183, 'epoch': 5.0})

In [35]:
#Wandb API Key: bcdbdd5ee9d76a20c90b5f2a246eb45f14b341a9
import os
os.environ["WANDB_DISABLED"] = "true"     #Disabling Wandb
#model.resize_token_embeddings(len(tokenizer))
trainer.train()

***** Running training *****
  Num examples = 126000
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 15750
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch,Training Loss,Validation Loss
1,0.669900,0.476513


Saving model checkpoint to models/hindi/T5_muRIL_3/checkpoint-10000
Configuration saved in models/hindi/T5_muRIL_3/checkpoint-10000/config.json
Model weights saved in models/hindi/T5_muRIL_3/checkpoint-10000/pytorch_model.bin
tokenizer config file saved in models/hindi/T5_muRIL_3/checkpoint-10000/tokenizer_config.json
Special tokens file saved in models/hindi/T5_muRIL_3/checkpoint-10000/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 14000
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=15750, training_loss=0.7122194562639509, metrics={'train_runtime': 4755.8781, 'train_samples_per_second': 26.494, 'train_steps_per_second': 3.312, 'total_flos': 5930269812449280.0, 'train_loss': 0.7122194562639509, 'epoch': 1.0})

In [35]:
#Wandb API Key: bcdbdd5ee9d76a20c90b5f2a246eb45f14b341a9
import os
os.environ["WANDB_DISABLED"] = "true"     #Disabling Wandb
#model.resize_token_embeddings(len(tokenizer))
trainer.train()

***** Running training *****
  Num examples = 126000
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 31500
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch,Training Loss,Validation Loss
1,1.246400,1.125347


Saving model checkpoint to models/hindi/T5_muRIL_3/checkpoint-2000
Configuration saved in models/hindi/T5_muRIL_3/checkpoint-2000/config.json
Model weights saved in models/hindi/T5_muRIL_3/checkpoint-2000/pytorch_model.bin
tokenizer config file saved in models/hindi/T5_muRIL_3/checkpoint-2000/tokenizer_config.json
Special tokens file saved in models/hindi/T5_muRIL_3/checkpoint-2000/special_tokens_map.json
Saving model checkpoint to models/hindi/T5_muRIL_3/checkpoint-4000
Configuration saved in models/hindi/T5_muRIL_3/checkpoint-4000/config.json
Model weights saved in models/hindi/T5_muRIL_3/checkpoint-4000/pytorch_model.bin
tokenizer config file saved in models/hindi/T5_muRIL_3/checkpoint-4000/tokenizer_config.json
Special tokens file saved in models/hindi/T5_muRIL_3/checkpoint-4000/special_tokens_map.json
Saving model checkpoint to models/hindi/T5_muRIL_3/checkpoint-6000
Configuration saved in models/hindi/T5_muRIL_3/checkpoint-6000/config.json
Model weights saved in models/hindi/T5_m

TrainOutput(global_step=31500, training_loss=1.3424053199404762, metrics={'train_runtime': 11415.222, 'train_samples_per_second': 11.038, 'train_steps_per_second': 2.759, 'total_flos': 5013459920793600.0, 'train_loss': 1.3424053199404762, 'epoch': 1.0})

In [37]:
#torch_device = 'cuda' if torch.cuda.is_available() else 'cpu' 

In [36]:
#Save the trained Model
trainer.save_model('t5_gec_hindi_muRIL')

Saving model checkpoint to t5_gec_hindi_muRIL
Configuration saved in t5_gec_hindi_muRIL/config.json
Model weights saved in t5_gec_hindi_muRIL/pytorch_model.bin
tokenizer config file saved in t5_gec_hindi_muRIL/tokenizer_config.json
Special tokens file saved in t5_gec_hindi_muRIL/special_tokens_map.json


In [ ]:
#!zip -r 't5_gec_hindi_muRIL_2.zip' 't5_gec_hindi_muRIL_2'

# Testing Trained Model

In [6]:
#Testing
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration,AutoTokenizer
model_name = 't5_gec_hindi_muRIL'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu' 

tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")


In [7]:
#tokenizer = AutoTokenizer.from_pretrained("surajp/RoBERTa-hindi-guj-san")
#tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")
trained_model = T5ForConditionalGeneration.from_pretrained(model_name).to(torch_device)

In [3]:
trained_model

T5ForConditionalGeneration(
  (shared): Embedding(197285, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(197285, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): D

In [4]:
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        param = parameter.numel()
        table.add_row([name, param])
        total_params+=param
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

count_parameters(trained_model)

+----------------------------------------------------------------------+------------+
|                               Modules                                | Parameters |
+----------------------------------------------------------------------+------------+
|                            shared.weight                             | 151514880  |
|            encoder.block.0.layer.0.SelfAttention.q.weight            |   589824   |
|            encoder.block.0.layer.0.SelfAttention.k.weight            |   589824   |
|            encoder.block.0.layer.0.SelfAttention.v.weight            |   589824   |
|            encoder.block.0.layer.0.SelfAttention.o.weight            |   589824   |
| encoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight |    384     |
|              encoder.block.0.layer.0.layer_norm.weight               |    768     |
|           encoder.block.0.layer.1.DenseReluDense.wi.weight           |  2359296   |
|           encoder.block.0.layer.1.DenseReluDense.wo.

349744128

In [5]:
param_size = 0
for param in trained_model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in trained_model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('model size: {:.3f}MB'.format(size_all_mb))

model size: 1334.168MB


In [8]:
def correct_grammar(input_text,num_return_sequences):
    #batch = tokenizer([input_text],truncation=True,padding='max_length',max_length=64, return_tensors="pt").to(torch_device)
    batch = tokenizer([input_text],truncation=True,padding='max_length',max_length=64, return_tensors="pt",return_token_type_ids=False).to(device)
    #print(batch)
    #translated = model1.generate(**batch,max_length=64,num_beams=4, num_return_sequences=num_return_sequences, temperature=1.5)
    translated= trained_model.generate(**batch,max_length=64,num_beams=4, num_return_sequences=num_return_sequences, temperature=1.5)
    #print("here")
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return tgt_text

In [78]:
#print(T5ForConditionalGeneration.__version__)

In [42]:
text = test_df['input'].iat[0]
correct = test_df['output'].iat[0]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
l = len(correct)
predicted_s = correct_grammar(text, num_return_sequences=4)[0]
print("Predicted Sentence:",predicted_s)

Incorrect Sentence: अँगरेज़ी सहभाषा के रूप में व्यवहत्त होनी चाहिए थी परंतु आज भी उसकी वर्चस्व सर्वत्र स्थापित है ।
Correct Seentence: अँगरेज़ी सहभाषा के रूप में व्यवहत्त होनी चाहिए थी परंतु आज भी उसका वर्चस्व सर्वत्र स्थापित है । 
Predicted Sentence: अँगरेज़ी सहभाषा के रूप में व्यवहारत होनी चाहिए थी परंतु आज भी उसकी वर्चस्व सर्वत्र स्थापित है ।


In [43]:
text = test_df['input'].iat[13]
correct = test_df['output'].iat[13]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
l = len(correct)
predicted_s = correct_grammar(text, num_return_sequences=2)[0]
p = len(predicted_s)
'''
if(p>l):
    predicted_s = predicted_s[:l]
elif (p<l-4):
    predicted_s = text
'''

print("Predicted Sentence:",predicted_s)

Incorrect Sentence: प्रत्येक शिक्षक जानता हैं कि उसके मासिक वेतन का कुछ अंश घूस में जा रहा हैं ।
Correct Seentence: प्रत्येक शिक्षक जानता है कि उसके मासिक वेतन का कुछ अंश घूस में जा रहा है । 
Predicted Sentence: प्रत्येक शिक्षक जानता है कि उसके मासिक वेतन का कुछ अंश घूस में जा रहा है ।


In [44]:
text = test_df['input'].iat[43]
correct = test_df['output'].iat[43]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
l = len(correct)
predicted_s = correct_grammar(text, num_return_sequences=2)[0]
if(p>l):
    predicted_s = predicted_s[:l]

print("Predicted Sentence:",predicted_s)

Incorrect Sentence: इस सबमें अहम बात मुस्लिम मतदाताओं का रुझान हैं ।
Correct Seentence: इस सबमें अहम बात मुस्लिम मतदाताओं का रुझान है । 
Predicted Sentence: इस सबमें अहम बात मुस्लिम मतदाताओं का रुझान है ।


In [45]:
text = test_df['input'].iat[1]
correct = test_df['output'].iat[1]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
l = len(correct)
predicted_s = correct_grammar(text, num_return_sequences=4)[0][:l]
print("Predicted Sentence:",predicted_s)

Incorrect Sentence: तथे गौशाला का
Correct Seentence: तथा गौशाला का
Predicted Sentence: तथा गौशाला का


In [46]:
text = test_df['input'].iat[0]
correct = test_df['output'].iat[0]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
l = len(correct)
predicted_s = correct_grammar(text, num_return_sequences=2)[0]
p = len(predicted_s)
if(p>l):
    predicted_s = predicted_s[:l]

print("Predicted Sentence:",predicted_s)

Incorrect Sentence: अँगरेज़ी सहभाषा के रूप में व्यवहत्त होनी चाहिए थी परंतु आज भी उसकी वर्चस्व सर्वत्र स्थापित है ।
Correct Seentence: अँगरेज़ी सहभाषा के रूप में व्यवहत्त होनी चाहिए थी परंतु आज भी उसका वर्चस्व सर्वत्र स्थापित है । 
Predicted Sentence: अँगरेज़ी सहभाषा के रूप में व्यवहारत होनी चाहिए थी परंतु आज भी उसकी वर्चस्व सर्वत्र स्थापित है ।


In [47]:
text = test_df['input'].iat[50]
correct = test_df['output'].iat[50]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
l = len(correct)
p = len(predicted_s)
if(p>l):
    predicted_s = predicted_s[:l]

print("Predicted Sentence:",predicted_s)

Incorrect Sentence: राष्ट्रीय जनता दल हरियाणा आज महिला आरक्षण विधेयक के विरोध में प्रदेश सचिव विजय सिंह कारगवाल के तृत्व में अंबेडकर पार्क के समीप जोदार प्रदर्शन किया तथा कांग्रेस अध्यक्षा सोनिया गांधी व भाजपा की वरिष्ठ त्री सुषमा स्वराज का पुतला फूंका ।
Correct Seentence: राष्ट्रीय जनता दल हरियाणा ने आज महिला आरक्षण विधेयक के विरोध में प्रदेश सचिव विजय सिंह कारगवाल के नेतृत्व में अंबेडकर पार्क के समीप जोदार प्रदर्शन किया तथा कांग्रेस अध्यक्षा सोनिया गांधी व भाजपा की वरिष्ठ नेत्री सुषमा स्वराज का पुतला फूंका । 
Predicted Sentence: राष्ट्रीय जनता दल हरियाणा आज महिला आरक्षण विधेयक के विरोध में प्रदेश सचिव विजय सिंह ने कारगवाल के नेतृत्व में अंबेडकर पार्क के समीप जोदार प्रदर्शन किया तथा कांग्रेस अध्यक्षा सोनिया गांधी व भाजपा की वरिष्ठ त्री सुषमा स्वराज का पुतला फूंका ।


In [48]:
text = test_df['input'].iat[60]
correct = test_df['output'].iat[60]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
l = len(correct)
p = len(predicted_s)
if(p>l):
    predicted_s = predicted_s[:l]
print("Predicted Sentence:",predicted_s)


Incorrect Sentence: ज़ाहिर है जब मैंने लिखूंगा तो वो मेरे ही व्यक्तिगत विचार कहलायेंगे किसी संस्थान के नहीं ।
Correct Seentence: ज़ाहिर है जब मैं लिखूंगा तो वो मेरे ही व्यक्तिगत विचार कहलायेंगे किसी संस्थान के नहीं । 
Predicted Sentence: ज़ाहिर है जब मैं लिखूंगा तो वो मेरे ही व्यक्तिगत विचार कहलायेंगे किसी संस्थान के नहीं ।


In [49]:
text = test_df['input'].iat[51]
correct = test_df['output'].iat[51]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
print("Predicted Sentence:",predicted_s)

Incorrect Sentence: गनीमत हैं गणेशशंकर विद्यार्थी ने २०० रुपये का मनीआर्डर भेजकर अशफ़ाक़ की मजार पर छत डलवा कर उसे धूप के साये से बचा लिया ।
Correct Seentence: गनीमत है गणेशशंकर विद्यार्थी ने २०० रुपये का मनीआर्डर भेजकर अशफ़ाक़ की मजार पर छत डलवा कर उसे धूप के साये से बचा लिया । 
Predicted Sentence: गनीमत है गणेशशंकर विद्यार्थी ने २०० रुपये का मनीआर्डर भेजकर अशफ़ाक़ की मजार पर छत डलवा कर उसे धूप के साये से बचा लिया ।


In [53]:
text = test_df['input'].iat[20]
correct = test_df['output'].iat[20]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
p=len(predicted_s)
l = len(text)
if(p>l):
    predicted_s = predicted_s[:l]
print("Predicted Sentence:",predicted_s)

Incorrect Sentence: हम पास खींच लेते हैंं ।
Correct Seentence: हम पास खींच लेते हैं । 
Predicted Sentence: हम पास खींच लेते हैं । 


# Performance Analysis
1. BLEU Score
2. GLEU Score


In [54]:
test_df.shape

(14000, 3)

In [55]:
test_df.head(5)

,input,output,input_token_len
40665,अँगरेज़ी सहभाषा के रूप में व्यवहत्त होनी चाहिए थी परंतु आज भी उसकी वर्चस्व सर्वत्र स्थापित है ।,अँगरेज़ी सहभाषा के रूप में व्यवहत्त होनी चाहिए थी परंतु आज भी उसका वर्चस्व सर्वत्र स्थापित है ।,28
48520,तथे गौशाला का,तथा गौशाला का,6
138403,बच्चे खेल रहे है किलकारियाँ गूंज रही है ब्रह्माण्ड में,बच्चे खेल रहे हैं किलकारियाँ गूंज रही हैं ब्रह्माण्ड में,14
130079,धरने पर किसानो ने कहा कि अब समय आ गया हैं कि गन्ने की खेती किसानो के लिये नुकसान का सौदा हो गयी हैं ।,धरने पर किसानो ने कहा कि अब समय आ गया है कि गन्ने की खेती किसानो के लिये नुकसान का सौदा हो गयी है ।,29
50146,सड़क मलबे का ढेर होने से दुपहिया व चार पहिया वाहन चालकों को परेशान होना पड़ रहा हैं ।,सड़क मलबे का ढेर होने से दुपहिया व चार पहिया वाहन चालकों को परेशान होना पड़ रहा है ।,23


In [9]:
import pandas as pd
df_test = pd.read_csv('DATA/etoori_test.csv')
df_test.shape

(30000, 2)

In [10]:
df_test.head(5)

,enc_input,dec_input
0,इसके अलावा माइकल शूमाकर द्वारा चलाई गई एक फरार...,इसके अलावा माइकल शूमाकर द्वारा चलाई गई एक फरार...
1,यह मन को काबू में करने वाली मुद्रा हैं इसीलिए ...,यह मन को काबू में करने वाली मुद्रा है इसीलिए इ...
2,आप पुस्तक पढ़ने में तल्लीन हैं और बच्चा उसनेाँ...,आप पुस्तक पढ़ने में तल्लीन हैं और बच्चा वहाँ प...
3,परिवार के मुताबिक धमाकों में हिस्सा लेने वाले ...,परिवार के मुताबिक धमाकों में हिस्सा लेने वाले ...
4,उनकी वो वाली बात भी अनिश्चित रहती हैं ।,उनकी वो वाली बात भी अनिश्चित रहती है ।


In [13]:
text = df_test['enc_input'].iat[0]
correct = df_test['dec_input'].iat[0]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
l = len(correct)
p = len(predicted_s)
if(p>l):
    predicted_s = predicted_s[:l]

print("Predicted Sentence:",predicted_s)

Incorrect Sentence: इसके अलावा माइकल शूमाकर द्वारा चलाई गई एक फरारी कार भी को मिलियन डॉलर क्लब में माना जा रहा है ।
Correct Seentence: इसके अलावा माइकल शूमाकर द्वारा चलाई गई एक फरारी कार को भी मिलियन डॉलर क्लब में माना जा रहा है । 
Predicted Sentence: इसके अलावा माइकल शूमाकर द्वारा चलाई गई एक फरारी कार को भी मिलियन डॉलर क्लब में माना जा रहा है ।


In [14]:
%%time
predicted_s = correct_grammar(text, num_return_sequences=1)[0]

CPU times: user 1.26 s, sys: 84.4 ms, total: 1.34 s
Wall time: 1.35 s


In [59]:
text = df_test['enc_input'].iat[1]
correct = df_test['dec_input'].iat[1]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
l = len(correct)
p = len(predicted_s)
if(p>l):
    predicted_s = predicted_s[:l]
print("Predicted Sentence:",predicted_s)

Incorrect Sentence: यह मन को काबू में करने वाली मुद्रा हैं इसीलिए इसे चित्त हस्त मुद्रा योग कहते हैंं ।
Correct Seentence: यह मन को काबू में करने वाली मुद्रा है इसीलिए इसे चित्त हस्त मुद्रा योग कहते हैं । 
Predicted Sentence: यह मन को काबू में करने वाली मुद्रा है इसीलिए इसे चित्त हस्त मुद्रा योग कहते हैं ।


In [60]:
text = df_test['enc_input'].iat[2]
correct = df_test['dec_input'].iat[2]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
l = len(correct)
p = len(predicted_s)
if(p>l):
    predicted_s = predicted_s[:l]
print("Predicted Sentence:",predicted_s)

Incorrect Sentence: आप पुस्तक पढ़ने में तल्लीन हैं और बच्चा उसनेाँ पर शोर मचा रहा है ।
Correct Seentence: आप पुस्तक पढ़ने में तल्लीन हैं और बच्चा वहाँ पर शोर मचा रहा है । 
Predicted Sentence: आप पुस्तक पढ़ने में तल्लीन हैं और बच्चा वहाँ पर शोर मचा रहा है ।


In [61]:
text = df_test['enc_input'].iat[3]
correct = df_test['dec_input'].iat[3]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
l = len(correct)
p = len(predicted_s)
if(p>l):
    predicted_s = predicted_s[:l]
print("Predicted Sentence:",predicted_s)

Incorrect Sentence: परिवार के मुताबिक धमाकों में हिस्सा लेने वाले कोई दूसरे लोग थे और ये गलत पहचान का मामला हैं ।
Correct Seentence: परिवार के मुताबिक धमाकों में हिस्सा लेने वाले कोई दूसरे लोग थे और ये गलत पहचान का मामला है । 
Predicted Sentence: परिवार के मुताबिक धमाकों में हिस्सा लेने वाले कोई दूसरे लोग थे और ये गलत पहचान का मामला है ।


In [62]:
text = df_test['enc_input'].iat[4]
correct = df_test['dec_input'].iat[4]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
l = len(correct)
p = len(predicted_s)
if(p>l):
    predicted_s = predicted_s[:l]
print("Predicted Sentence:",predicted_s)

Incorrect Sentence: उनकी वो वाली बात भी अनिश्चित रहती हैं ।
Correct Seentence: उनकी वो वाली बात भी अनिश्चित रहती है । 
Predicted Sentence: उनकी वो वाली बात भी अनिश्चित रहती है ।


In [64]:
text = df_test['enc_input'].iat[9]
correct = df_test['dec_input'].iat[9]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
l = len(correct)
p = len(predicted_s)
if(p>l):
    predicted_s = predicted_s[:l]
print("Predicted Sentence:",predicted_s)

Incorrect Sentence: मैंने ललित हूँ ।
Correct Seentence: मैं ललित हूँ । 
Predicted Sentence: मैं ललित हूँ । 


In [ ]:
import nltk.translate.bleu_score as bleu
BLEU = []
index = []
test_data = df_test.head(10000)
np.random.seed(1)
for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        text = str(i.enc_input)
        #pred = predict(str(i.enc_input),model)[0].split()
        pred = correct_grammar(text, num_return_sequences=1)[0]
        act = [str(i.dec_input).split()]
        pred_s = str(pred).split()
        #print(act)
        #print(pred_s)
        b = bleu.sentence_bleu(act,pred_s)
        BLEU.append(b)
    except:
        index.append(ind)
        continue
print("BELU = ", np.mean(BLEU))

In [63]:
from nltk.translate.gleu_score import sentence_gleu
GLEU_val_emb = []
test_data = df_test.head(10)
print(test_data.shape)
for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        text = str(i.enc_input)
        #pred = predict(str(i.enc_input),model)[0].split()
        pred = correct_grammar(text, num_return_sequences=1)[0]
        l = len(text)
        p = len(pred)
        if(p>l):
            pred = pred[:l]
            
        act = [str(i.dec_input).split()]
        pred_s = str(pred).split()
    
        #print(act)
        #print(pred_s)
        b = sentence_gleu(act,pred_s)
        #print(b)
        GLEU_val_emb.append(b)
    except:
        continue
print("GELU Score = ",np.mean(GLEU_val_emb))

(10, 2)


10it [00:13,  1.38s/it]

GELU Score =  0.8984656084656084


In [64]:
from nltk.translate.gleu_score import sentence_gleu
GLEU_val_emb = []
test_data = df_test.head(10000)
print(test_data.shape)
for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        text = str(i.enc_input)
        #pred = predict(str(i.enc_input),model)[0].split()
        pred = correct_grammar(text, num_return_sequences=1)[0]
        l = len(text)
        p = len(pred)
        if(p>l):
            pred = pred[:l]
            
        act = [str(i.dec_input).split()]
        pred_s = str(pred).split()
        b = sentence_gleu(act,pred_s)
        GLEU_val_emb.append(b)
        if(ind%1000 ==0):
            print("GELU Score for ",ind+1," data point",np.mean(GLEU_val_emb))
    except:
        continue
print("GELU Score = ",np.mean(GLEU_val_emb))

(10000, 2)


1it [00:01,  1.41s/it]

GELU Score for  1  data point 1.0


1001it [21:58,  1.20s/it]

GELU Score for  1001  data point 0.9150283140478553


...

GELU Score for  8001  data point 0.9201028812376454


9001it [2:53:42,  1.13s/it]

GELU Score for  9001  data point 0.9205000001846352


10000it [3:12:33,  1.16s/it]

GELU Score =  0.9208625332430048


In [65]:
print("GELU Score = ",np.mean(GLEU_val_emb))

GELU Score =  0.9208625332430048


In [71]:
FScore_val_emb = []
test_data = df_test.head(10000)
print(test_data.shape)

for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        #pred = predict(str(i.enc_input),model)[0].split()
        #act = [str(i.dec_output).split()]
        
        act = str(i.dec_input)
        #pred = predict(str(i.enc_input),model)[0]
        text = str(i.enc_input)
        pred = correct_grammar(text, num_return_sequences=1)[0]
        
        #b =sentence_gleu(act,pred)
        recall,precision,f_beta = rouge_n(act,pred)
        #print(recall,precision,f_beta)
        
        #print("BELU Score",b)
        FScore_val_emb.append(f_beta)
    except:
        continue
print("F0.5 Score = ",np.mean(FScore_val_emb))

(10000, 2)


10000it [3:10:43,  1.14s/it]

F0.5 Score =  0.9521005068789202


In [72]:
import numpy as np
print("F0.5 Score = ",np.mean(FScore_val_emb))

F0.5 Score =  0.9521005068789202
